In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
#@title Imports. {display-mode:'form'}
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense,GlobalAveragePooling2D,Flatten,concatenate,BatchNormalization, Dropout
from tensorflow.keras.applications import InceptionV3,DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Visualize the Train/Val loss
import matplotlib.pyplot as plt
%matplotlib inline

## Set the data generators
### Loading the dataframes

In [ ]:
gen_path = '../../../../../../Datasets/Parkinson/radiological/PPMI/spect-mri/filtered/' 
    
csv_train = os.path.join(gen_path + 'control_pd_SPECT_fullRois_TRAIN.csv')
csv_test = os.path.join(gen_path + 'control_pd_SPECT_fullRois_TEST.csv')

# Load the training data from the CSV file and assign column names to the DataFrame
train_df = pd.read_csv(csv_train, header=None)
train_df.columns = ['path', 'label']

# Load the test data from the CSV file and assign column names to the DataFrame
test_df = pd.read_csv(csv_test, header=None)
test_df.columns = ['path', 'label']

In [ ]:
print(train_df.groupby('label').count())
print(test_df.groupby('label').count())

In [ ]:
#@title Set the data generators. {display-mode:'form', run: "auto"}
#@markdown Data augmentation choices. Cell runs automatically if anything is changed.
shear_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
zoom_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
width_shift_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
height_shift_range = 0.1 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
rotation_range = 10 #@param {type:"slider", min:0, max:90, step:5}
horizontal_flip = True #@param {type:"boolean"}
vertical_flip = False #@param {type:"boolean"}
#@markdown Data source (No need to change if the download succeeded.)


train_datagen = ImageDataGenerator(rescale=1./255,
                                    shear_range=shear_range,
                                    zoom_range=zoom_range,
                                    width_shift_range=width_shift_range,
                                    height_shift_range=height_shift_range,
                                    rotation_range=rotation_range,
                                    horizontal_flip=horizontal_flip,
                                    vertical_flip=vertical_flip)

train_generator = train_datagen.flow_from_dataframe(directory=None,
                                                    dataframe=train_df,
                                                    x_col='path', 
                                                    y_col='label',
                                                    target_size=(256,256),
                                                    color_mode='rgb',
                                                    batch_size=8,
                                                    class_mode='categorical',
                                                    shuffle=True)

# Data Generator for validation without data augmentation!
val_datagen   = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(directory=None,
                                                 dataframe=test_df,
                                                 x_col='path', 
                                                 y_col='label',
                                                 target_size=(256,256),
                                                 color_mode='rgb',
                                                 batch_size=8,
                                                 class_mode='categorical',
                                                 shuffle=True)

## Set up the pretrained model and add dense layers

In [ ]:
#@title Set up the pretrained model, and add dense layers. {display-mode:'form', run: "auto"}
#@markdown Set up the trainable dense layers. Further options include BatchNorm (provides regularization), DropOut (also, for normalization, but should not be used together with BatchNorm), and GlobalAveragePooling as an alternative to simple flattening. Did not work well in our experiments.  Cell runs automatically if anything is changed.
first_dense_layer_neurons  = 1024 #@param {type:"integer"}
second_dense_layer_neurons = 256 #@param {type:"integer"}
use_global_average_pooling = False #@param {type:"boolean"}
use_batch_norm             = True #@param {type:"boolean"}
use_drop_out               = False  #@param {type:"boolean"}
pretrained_model           = 'Inception V3' #@param ["Inception V3", "DenseNet 121"]
optimizer                  = 'adam' #@param ['adam', 'adagrad', 'adadelta', 'sgd'] {allow-input: true}

if pretrained_model == 'Inception V3':
    base_model=InceptionV3(weights='imagenet',include_top=False, input_shape=(256,256,3))
else:
    base_model=DenseNet121(weights='imagenet',include_top=False, input_shape=(256,256,3))

x=base_model.output

if use_global_average_pooling == True:
    x=GlobalAveragePooling2D()(x)
else:
    x=Flatten()(x)

if use_batch_norm:
    x = BatchNormalization()(x)
if use_drop_out:
    x = Dropout(rate=0.5)(x)
x = Dense(first_dense_layer_neurons,activation='relu')(x)

if use_batch_norm:
    x = BatchNormalization()(x)
if use_drop_out:
    x = Dropout(rate=0.5)(x)
x = Dense(second_dense_layer_neurons,activation='relu')(x)

if use_batch_norm:
    x = BatchNormalization()(x)
if use_drop_out:
    x = Dropout(rate=0.5)(x)
preds = Dense(2,activation='softmax')(x) # final layer with softmax activation

model = Model(inputs=base_model.input,outputs=preds)

### First pass: train addedd dense layers
First train only the top layers (randomly initialized) freezing all convolutional InceptionV3 layers

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

if optimizer in ['adam', 'adagrad', 'adadelta', 'sgd']: # standard settings
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics = ['accuracy']) # categorical crossentropy would also do...
else:
    from tensorflow.keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=train_generator.n//train_generator.batch_size,
                              epochs=30, # Originally, 500 epochs!
                             validation_data=val_generator,
                             validation_steps=val_generator.n//val_generator.batch_size)

In [ ]:
#@title Plot train and validation loss/accuracy {display-mode:'form'}

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# Depending on the version of TF/Keras, the metric is either stored as 'acc' or 'accuracy'. This is not checked here.
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

### Fine-tune last convolutional layers
The added layers should have converged. It is a good practice to fine-tune the top convolutional layers. 

In this case we chose to fine-tune the top 2 inception blocks, i.e. we will freeze the first 249 layers and unfreeze the rest. Afterwards, the model needs to be recompiled.

Note that this will not change the trained parameters.

In [ ]:
#@title Set up trainable parameters {display-mode:'form'}
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True
    
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history_finetune = model.fit_generator(generator=train_generator,
                                       steps_per_epoch=train_generator.n//train_generator.batch_size,
                                       epochs=20,
                                       validation_data=val_generator,
                                       validation_steps=val_generator.n//val_generator.batch_size)

In [ ]:
#@title Plot train and validation loss/accuracy {display-mode:'form'}

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.show()

# Depending on the version of TF/Keras, the metric is either stored as 'acc' or 'accuracy'. This is not checked here.
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#Confution Matrix and Classification Report
val_generator.reset()
logits = model.predict(val_generator)
y_pred_class = np.argmax(logits, axis=1)
#predicted_class_probab=np.max(logits,axis=1)

target_names = ['control', 'parkinson']   
  
print('Confusion Matrix')
print(confusion_matrix(val_generator.classes, y_pred_class))
print('Classification Report')
print(classification_report(val_generator.classes, y_pred_class, target_names=target_names))

In [ ]:
model.summary()